<a href="https://colab.research.google.com/github/Best-Island-LCH/AI_NLP_Team08/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI 품질 평가 모델 학습 - klue/roberta-base

이 노트북은 AI 응답 품질 평가를 위한 Multi-label Classification 모델을 학습합니다.

**평가 기준 (9개)**:

- linguistic_acceptability (언어적 수용성)
- consistency (일관성)
- interestingness (흥미로움)
- unbias (편향 없음)
- harmlessness (무해성)
- no_hallucination (환각 없음)
- understandability (이해 가능성)
- sensibleness (합리성)
- specificity (구체성)

## 1. 라이브러리 설치 및 임포트

In [ ]:
!pip install torch transformers datasets pandas scikit-learn tqdm -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## 2. 설정

In [ ]:
# 모델 및 학습 설정
MODEL_NAME = "klue/roberta-base"
MAX_LENGTH = 256
BATCH_SIZE = 64
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
SEED = 42

# 평가 기준 (타겟 컬럼)
CRITERIA = [
    'linguistic_acceptability',
    'consistency',
    'interestingness',
    'unbias',
    'harmlessness',
    'no_hallucination',
    'understandability',
    'sensibleness',
    'specificity'
]
NUM_LABELS = len(CRITERIA)

# 시드 고정
torch.manual_seed(SEED)
np.random.seed(SEED)

## 3. 데이터 로드

In [ ]:
# # unzip
# unzip_path = '/content/drive/MyDrive/mutsa-01/data.zip'
# !unzip -qq {unzip_path} -d /content/

In [ ]:
# Training 및 Validation 데이터 로드 (aggregated 버전 사용 - majority voting 결과)
train_df = pd.read_csv('/content/drive/MyDrive/mutsa-01/data/train/training_all_aggregated.csv', encoding='utf-8-sig')
val_df = pd.read_csv('/content/drive/MyDrive/mutsa-01/data/val/validation_all_aggregated.csv', encoding='utf-8-sig')

In [ ]:
train_df = train_df.iloc[:10000]
val_df = val_df.iloc[:1200]

In [ ]:
print(f"Training 데이터: {len(train_df):,}개")
print(f"Validation 데이터: {len(val_df):,}개")

Training 데이터: 10,000개
Validation 데이터: 1,200개


In [ ]:
# 데이터 확인
train_df.head()

,source_file,conversation_id,topic,num_evaluators,exchange_id,utterance_id,utterance_index,num_evaluations,human_question,bot_response,...,understandability_majority,understandability_unanimous,sensibleness_yes_count,sensibleness_no_count,sensibleness_majority,sensibleness_unanimous,specificity_yes_count,specificity_no_count,specificity_majority,specificity_unanimous
0,경제활동_상품상거래_209.json,864,"경제활동, 상품/상거래",5,c864.e1,c864.u2,1,3,"저번 주 포스코홀딩스의 실적이 나왔는데, 어떤 내용이었어?",포스코홀딩스의 3분기 실적은 철강시황 악화와 타이푼 힌남노로 인한 침수 피해로 인해...,...,1,1,3,0,1,1,1,2,0,0
1,경제활동_상품상거래_209.json,864,"경제활동, 상품/상거래",5,c864.e2,c864.u4,3,3,영업이익도 많이 감소했으려나?,영업이익은 71% 감소하여 9천억원이었습ㄴㅣ다.,...,1,1,3,0,1,1,3,0,1,1
2,경제활동_상품상거래_209.json,864,"경제활동, 상품/상거래",5,c864.e3,c864.u6,5,3,왜 영업이익이 감소 원인은 뭐라고 지목되고 있는데?,영업이익 감소의 원인으로는 철강 업황 악화와 태풍 힌남로로 인한 침수 피해 등이 꼽...,...,1,1,3,0,1,1,3,0,1,1
3,경제활동_상품상거래_209.json,864,"경제활동, 상품/상거래",5,c864.e4,c864.u8,7,3,"포스코홀딩스는 이전 분기에 좋은 실적을 올렸었는데, 왜 실적이 감소했을까?",올해 3분기부터 세계 경기 침체 우려가 커지면서 가전 수요 등이 줄어 철강 가격도 ...,...,1,1,3,0,1,1,3,0,1,1
4,경제활동_상품상거래_209.json,864,"경제활동, 상품/상거래",5,c864.e5,c864.u10,9,3,"포항제철소가 침수돼 생산이 중단되었다고 들었는데, 이로 인한 영향이 있었겠네.","네, 포항제철소의 생산 중단으로 인한 침수 피해로 영업이익 감소 폭이 4400억원으...",...,1,1,3,0,1,1,3,0,1,1


In [ ]:
# 컬럼 확인
print("컬럼 목록:")
print(train_df.columns.tolist())

컬럼 목록:
['source_file', 'conversation_id', 'topic', 'num_evaluators', 'exchange_id', 'utterance_id', 'utterance_index', 'num_evaluations', 'human_question', 'bot_response', 'bot_response_length', 'linguistic_acceptability_yes_count', 'linguistic_acceptability_no_count', 'linguistic_acceptability_majority', 'linguistic_acceptability_unanimous', 'consistency_yes_count', 'consistency_no_count', 'consistency_majority', 'consistency_unanimous', 'interestingness_yes_count', 'interestingness_no_count', 'interestingness_majority', 'interestingness_unanimous', 'unbias_yes_count', 'unbias_no_count', 'unbias_majority', 'unbias_unanimous', 'harmlessness_yes_count', 'harmlessness_no_count', 'harmlessness_majority', 'harmlessness_unanimous', 'no_hallucination_yes_count', 'no_hallucination_no_count', 'no_hallucination_majority', 'no_hallucination_unanimous', 'understandability_yes_count', 'understandability_no_count', 'understandability_majority', 'understandability_unanimous', 'sensibleness_yes_count

## 4. 데이터 전처리

In [ ]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """데이터 전처리: 입력 텍스트 생성 및 결측치 처리"""
    df = df.copy()

    # 결측치 처리
    df['human_question'] = df['human_question'].fillna('')
    df['bot_response'] = df['bot_response'].fillna('')

    # 입력 텍스트 생성: [질문] + [SEP] + [응답]
    df['input_text'] = df['human_question'] + ' [SEP] ' + df['bot_response']

    # 타겟 컬럼 추출 (majority voting 결과)
    target_cols = [f'{c}_majority' for c in CRITERIA]

    # 타겟 결측치 제거
    df = df.dropna(subset=target_cols)

    return df

train_df = preprocess_data(train_df)
val_df = preprocess_data(val_df)

print(f"전처리 후 Training 데이터: {len(train_df):,}개")
print(f"전처리 후 Validation 데이터: {len(val_df):,}개")

전처리 후 Training 데이터: 10,000개
전처리 후 Validation 데이터: 1,200개


In [ ]:
# 레이블 분포 확인
target_cols = [f'{c}_majority' for c in CRITERIA]

print("=" * 50)
print("레이블 분포 (Training)")
print("=" * 50)
for col in target_cols:
    pos_ratio = train_df[col].mean()
    print(f"{col}: {pos_ratio:.2%} positive")

레이블 분포 (Training)
linguistic_acceptability_majority: 85.32% positive
consistency_majority: 86.49% positive
interestingness_majority: 87.56% positive
unbias_majority: 89.33% positive
harmlessness_majority: 88.66% positive
no_hallucination_majority: 78.10% positive
understandability_majority: 86.15% positive
sensibleness_majority: 84.96% positive
specificity_majority: 89.46% positive


## 5. Tokenizer 로드

In [ ]:
# klue/roberta-base 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"Tokenizer 로드 완료: {MODEL_NAME}")
print(f"Vocab size: {tokenizer.vocab_size:,}")

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Tokenizer 로드 완료: klue/roberta-base
Vocab size: 32,000


In [ ]:
# 토크나이저 테스트
sample_text = train_df['input_text'].iloc[0]
print(f"샘플 텍스트:\n{sample_text}\n")

tokens = tokenizer(sample_text, truncation=True, max_length=MAX_LENGTH)
print(f"토큰 수: {len(tokens['input_ids'])}")

샘플 텍스트:
저번 주 포스코홀딩스의 실적이 나왔는데, 어떤 내용이었어? [SEP] 포스코홀딩스의 3분기 실적은 철강시황 악화와 타이푼 힌남노로 인한 침수 피해로 인해 71% 급감했습니다.

토큰 수: 54


## 6. Dataset 클래스 정의

In [ ]:
class QualityEvalDataset(Dataset):
    """AI 품질 평가 데이터셋"""

    def __init__(self, df: pd.DataFrame, tokenizer, max_length: int = 256):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.target_cols = [f'{c}_majority' for c in CRITERIA]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['input_text']
        labels = row[self.target_cols].values.astype(np.float32)

        # 토크나이즈
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [ ]:
# Dataset 생성
train_dataset = QualityEvalDataset(train_df, tokenizer, MAX_LENGTH)
val_dataset = QualityEvalDataset(val_df, tokenizer, MAX_LENGTH)

print(f"Train dataset size: {len(train_dataset):,}")
print(f"Val dataset size: {len(val_dataset):,}")

Train dataset size: 10,000
Val dataset size: 1,200


In [ ]:
# Dataset 확인
sample = train_dataset[0]
print(f"input_ids shape: {sample['input_ids'].shape}")
print(f"attention_mask shape: {sample['attention_mask'].shape}")
print(f"labels shape: {sample['labels'].shape}")
print(f"labels: {sample['labels']}")

input_ids shape: torch.Size([256])
attention_mask shape: torch.Size([256])
labels shape: torch.Size([9])
labels: tensor([0., 1., 0., 1., 1., 1., 1., 1., 0.])


## 7. 모델 로드

In [ ]:
# Multi-label Classification을 위한 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

model.to(device)
print(f"모델 로드 완료: {MODEL_NAME}")
print(f"파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델 로드 완료: klue/roberta-base
파라미터 수: 110,625,033


## 8. 평가 지표 함수

In [ ]:
def compute_metrics(eval_pred):
    """평가 지표 계산"""
    predictions, labels = eval_pred

    # Sigmoid 적용 후 0.5 기준으로 이진화
    predictions = torch.sigmoid(torch.tensor(predictions)).numpy()
    predictions = (predictions > 0.5).astype(int)
    labels = labels.astype(int)

    # 전체 정확도 (모든 레이블이 일치해야 정답)
    exact_match = np.all(predictions == labels, axis=1).mean()

    # 레이블별 정확도
    per_label_acc = (predictions == labels).mean(axis=0)

    # Micro/Macro F1
    micro_f1 = f1_score(labels, predictions, average='micro')
    macro_f1 = f1_score(labels, predictions, average='macro')

    metrics = {
        'exact_match': exact_match,
        'micro_f1': micro_f1,
        'macro_f1': macro_f1,
    }

    # 각 기준별 정확도 추가
    for i, criterion in enumerate(CRITERIA):
        metrics[f'{criterion}_acc'] = per_label_acc[i]

    return metrics

## 9. Trainer 설정 및 학습

In [ ]:
# Training Arguments 설정
training_args = TrainingArguments(
    output_dir='./outputs/klue-roberta-base',
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True,
    report_to='none',  # wandb 사용 시 'wandb'로 변경
    seed=SEED,
    fp16=torch.cuda.is_available(),  # CUDA 사용 시 FP16 활성화
)

In [ ]:
# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# 학습 시작
print("학습 시작...")
train_result = trainer.train()
print("학습 완료!")

학습 시작...


Epoch,Training Loss,Validation Loss,Exact Match,Micro F1,Macro F1,Linguistic Acceptability Acc,Consistency Acc,Interestingness Acc,Unbias Acc,Harmlessness Acc,No Hallucination Acc,Understandability Acc,Sensibleness Acc,Specificity Acc
1,0.432900,0.244607,0.580833,0.954314,0.953965,0.913333,0.946667,0.901667,0.975833,0.974167,0.856667,0.875000,0.913333,0.917500
2,0.216400,0.226005,0.605000,0.956490,0.956376,0.915000,0.956667,0.903333,0.974167,0.975833,0.888333,0.866667,0.912500,0.918333
3,0.196500,0.218156,0.617500,0.958515,0.958484,0.915000,0.957500,0.904167,0.978333,0.980000,0.897500,0.870833,0.917500,0.920000


학습 완료!


## 10. 평가

In [ ]:
# Validation 데이터로 평가
eval_results = trainer.evaluate()

print("=" * 50)
print("평가 결과")
print("=" * 50)
for key, value in eval_results.items():
    if 'loss' in key or 'f1' in key or 'match' in key:
        print(f"{key}: {value:.4f}")

평가 결과
eval_loss: 0.2182
eval_exact_match: 0.6175
eval_micro_f1: 0.9585
eval_macro_f1: 0.9585


In [ ]:
# 기준별 정확도 출력
print("\n" + "=" * 50)
print("기준별 정확도")
print("=" * 50)
for criterion in CRITERIA:
    key = f'eval_{criterion}_acc'
    if key in eval_results:
        print(f"{criterion}: {eval_results[key]:.4f}")


기준별 정확도
linguistic_acceptability: 0.9150
consistency: 0.9575
interestingness: 0.9042
unbias: 0.9783
harmlessness: 0.9800
no_hallucination: 0.8975
understandability: 0.8708
sensibleness: 0.9175
specificity: 0.9200


## 11. 모델 저장

In [ ]:
# 모델 저장
save_path = '/content/drive/MyDrive/mutsa-01/outputs/klue-roberta-base-final'
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print(f"모델 저장 완료: {save_path}")

모델 저장 완료: /content/drive/MyDrive/mutsa-01/outputs/klue-roberta-base-final


## 12. 추론 예시

In [ ]:
def predict(text: str, model, tokenizer, device):
    """단일 텍스트에 대한 예측 수행"""
    model.eval()

    encoding = tokenizer(
        text,
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        return_tensors='pt'
    )

    encoding = {k: v.to(device) for k, v in encoding.items()}

    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]
        preds = (probs > 0.5).astype(int)

    results = {}
    for i, criterion in enumerate(CRITERIA):
        results[criterion] = {
            'prediction': int(preds[i]),
            'probability': float(probs[i])
        }

    return results

In [ ]:
# 추론 테스트
sample_question = "한국의 수도는 어디야?"
sample_response = "한국의 수도는 서울입니다. 서울은 대한민국의 정치, 경제, 문화의 중심지로, 약 1000만 명의 인구가 거주하고 있습니다."
sample_input = f"{sample_question} [SEP] {sample_response}"

print(f"질문: {sample_question}")
print(f"응답: {sample_response}")
print("\n" + "=" * 50)
print("예측 결과")
print("=" * 50)

results = predict(sample_input, model, tokenizer, device)
for criterion, values in results.items():
    status = "✓" if values['prediction'] == 1 else "✗"
    print(f"{status} {criterion}: {values['probability']:.2%}")

질문: 한국의 수도는 어디야?
응답: 한국의 수도는 서울입니다. 서울은 대한민국의 정치, 경제, 문화의 중심지로, 약 1000만 명의 인구가 거주하고 있습니다.

예측 결과
✓ linguistic_acceptability: 96.10%
✓ consistency: 97.87%
✓ interestingness: 95.81%
✓ unbias: 98.54%
✓ harmlessness: 98.81%
✓ no_hallucination: 95.59%
✓ understandability: 91.60%
✓ sensibleness: 96.14%
✓ specificity: 96.65%


In [ ]:
print(f"GPU 메모리 사용 전: {torch.cuda.memory_allocated() / (1024**3):.2f} GB")

# 1. 모델을 CPU로 이동
model.to('cpu')

# 2. Trainer 객체에서 model 참조 제거 (garbage collection)
del trainer.model
del trainer

# 3. 모델 변수 삭제
del model

# 4. PyTorch CUDA 캐시 비우기
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"GPU 메모리 사용 후: {torch.cuda.memory_allocated() / (1024**3):.2f} GB")


GPU 메모리 사용 전: 1.26 GB
GPU 메모리 사용 후: 0.85 GB
